In [1]:
import numpy as np
import scipy as sp
import matplotlib as mpl
import pandas as pd
import csv
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
from scipy import integrate
from lmfit import minimize, Parameters, Parameter, report_fit
from sklearn import datasets
from sklearn.linear_model import LinearRegression
import random
from shapely.geometry import Polygon
from shapely.geometry import Point


In [2]:
def calculate_iou(box_1, box_2):
    poly_1 = Polygon(box_1)
    poly_2 = Polygon(box_2)
    iou = poly_2.intersection(poly_1).area / poly_1.union(poly_2).area
    proportion=poly_2.area/(poly_1.union(poly_2).area)
    return (iou/proportion)

In [3]:
sizematrix=pd.read_csv('sizematrix.csv',header=None)

In [4]:
diametermatrix=sizematrix*2
diametermatrix

,0
0,2.000
1,2.018
2,2.036
3,2.054
4,2.072
...,...
2996,1992.800
2997,1994.600
2998,1996.400
2999,1998.200


In [5]:
particledata=pd.read_csv('CW32_____RAW_01-12-22_11.40.36.AM.csv')
##particledata.rename(columns = {'BOUNDING RECT HEIGHT':'minordiameter', 'MAX FERET DIAMETER':'majordiameter'}, inplace = True)
particledata.rename(columns = {'MINOR DIAMETER':'minordiameter', 'MAJOR DIAMETER':'majordiameter'}, inplace = True)
particledata

,CLASS,DATE,TIME,FRAME,minordiameter,majordiameter,ASPECT RATIO,BOUNDING RECT WIDTH,BOUNDING RECT HEIGHT,BOUNDING RECT DIAGONAL,...,RATIO OF EQUIVALENT RECT SIDES,ELONGATION FACTOR,COMPACTNESS FACTOR,HEYWOOD CIRCULARITY FACTOR,TYPE FACTOR,CIRCULARITY,SPHERICITY,CONVEXITY,Unnamed: 46,Unnamed: 47
0,NaN,01/12/22,11:39:16,74,672.57,861.55,1.28,787.22,686.65,1044.60,...,1.0,1.53,0.79,1.02,0.99,0.98,0.97,0.98,0.0,0.0
1,NaN,01/12/22,11:39:39,597,673.29,838.88,1.25,794.10,715.20,1068.69,...,1.0,1.41,0.79,1.01,1.00,0.99,0.98,0.98,0.0,0.0
2,NaN,01/12/22,11:39:41,627,647.27,814.57,1.26,735.39,671.75,996.02,...,1.0,1.43,0.71,1.01,0.99,0.99,0.97,0.98,0.0,0.0
3,NaN,01/12/22,11:39:42,653,660.34,837.20,1.27,833.01,657.64,1061.32,...,1.0,1.60,0.74,1.01,0.99,0.99,0.97,0.98,0.0,0.0
4,NaN,01/12/22,11:39:42,653,656.50,811.07,1.24,787.40,668.38,1032.82,...,1.0,1.47,0.80,1.01,0.99,0.99,0.98,0.98,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9014,NaN,01/12/22,11:46:36,9790,477.65,594.17,1.24,577.19,496.74,761.51,...,1.0,1.46,0.78,1.01,0.99,0.99,0.98,0.97,0.0,0.0
9015,NaN,01/12/22,11:46:37,9808,428.08,574.94,1.34,579.83,431.34,722.67,...,1.0,1.69,0.74,1.02,0.96,0.98,0.96,0.97,0.0,0.0
9016,NaN,01/12/22,11:46:38,9830,662.56,858.04,1.30,832.54,680.32,1075.16,...,1.0,1.60,0.77,1.02,0.98,0.98,0.97,0.98,0.0,0.0
9017,NaN,01/12/22,11:46:38,9837,619.21,795.15,1.28,787.63,617.30,1000.71,...,1.0,1.57,0.75,1.02,0.98,0.98,0.97,0.98,0.0,0.0


In [6]:
particledata.minordiameter=particledata.minordiameter
particledata.majordiameter=particledata.majordiameter
particledata.AREA=particledata.AREA
areaeqdiameter=np.array(np.sqrt(4*particledata.AREA/np.pi))
areaeqdiameter

array([761.21699719, 751.53779815, 726.11878882, ..., 753.99035586,
       701.68492356, 653.68126118])

In [7]:
primary=np.empty((len(areaeqdiameter),1))
secondary=np.empty((len(areaeqdiameter),1))
for q in range (0,len(areaeqdiameter),1):
    #a=(np.pi-2)/8
    #b=(majordiameter[q]-minordiameter[q])
    #c=((4*majordiameter[q]*minordiameter[q]+(np.pi-2)*minordiameter[q]**2)/8)-area[q]
    #x_rc[q]=(-b+np.sqrt((b**2)-4*a*c))/(2*a)
    primary[q]=((particledata.majordiameter[q])**2)/(2*particledata.minordiameter[q])
    secondary[q]=(particledata.minordiameter[q]**2)/(2*particledata.majordiameter[q])
primary=primary.flatten()
secondary=secondary.flatten()

In [8]:
primary

array([551.81498023, 522.59773233, 512.55603141, ..., 555.59695846,
       510.54046487, 499.16118634])

In [9]:
secondary

array([262.52127265, 270.1932482 , 257.16540807, ..., 255.80727798,
       241.09980765, 214.00928352])

In [10]:
x_rc=np.empty((len(areaeqdiameter),1))
for q in range (0,len(areaeqdiameter),1):
    a=(np.pi-2)/8
    b=(particledata.majordiameter[q]-particledata.minordiameter[q])
    c=((4*particledata.majordiameter[q]*particledata.minordiameter[q]+(np.pi-2)*particledata.minordiameter[q]**2)/8)-particledata.AREA[q]
    x_rc[q]=(-b+np.sqrt((b**2)-4*a*c))/(2*a)
x_rc=x_rc.flatten()
x_rc

array([407.887268  , 426.24086822, 403.3333878 , ..., 395.41009613,
       374.14863614, 327.60609699])

In [11]:
diameters=np.empty((len(x_rc),1))
for n in range (0,len(x_rc),1):
    total=int(primary[n])+int(secondary[n])
    binom=random.randint(0, total)
    if binom <= int(primary[n]):
        diameters[n]=primary[n]
    else: 
        diameters[n]=secondary[n]
diameters

array([[551.81498023],
       [522.59773233],
       [512.55603141],
       ...,
       [255.80727798],
       [510.54046487],
       [214.00928352]])

In [12]:
average=np.average(particledata.minordiameter)
average

640.2015977381085

In [13]:
indices=np.empty((len(particledata),1))
for n in range (0,len(indices),1):
    array = np.asarray(diametermatrix)
    value=diameters[n]
    idx = (np.abs(array - value)).argmin()
    indices[n]=idx
indices

array([[2195.],
       [2179.],
       [2174.],
       ...,
       [2031.],
       [2173.],
       [2008.]])

In [14]:
contactradiii=np.empty((len(indices),1))
for g in range (0,len(indices),1):
    diametermatrix2=np.array(diametermatrix)
    contactradiii[g]=diametermatrix2[int(indices[g])]
contactradiii

array([[551. ],
       [522.2],
       [513.2],
       ...,
       [255.8],
       [511.4],
       [214.4]])

In [15]:
totalelements=np.empty((3000,1))
for x in range(1,3001,1):
    with open(r'C:\Users\lfeltner\Dropbox\CSEE\ParticleSelector\UpdatedParticles\shot{}.inp'.format("{:04d}".format(x))) as p:
        for line in p:
            pass
        last_line = line
        totalelements[x-1]=int(last_line[0:4])

In [16]:
length = 5
length

5

In [17]:
width=5
width

5

In [18]:
'''
coverage=float(input("Coverage (%): "))/100
coverage
'''

'\ncoverage=float(input("Coverage (%): "))/100\ncoverage\n'

In [19]:
massflowrate_kg=float(input("Mass flow rate (kg/s):"))
## 0.04725 0.0945 0.189 kg/s

Mass flow rate (kg/s): 0.0945


In [20]:
peeningtime=float(input("Peening time (s):"))
## 40 s

Peening time (s): 20


In [21]:
partarea=float(input("Total Peened Part Area (mm^2):"))
## 23300 mm^2 or 49100 mm^2

Total Peened Part Area (mm^2): 49100


In [24]:
velo=float(input("Velocity (mm/s):"))

Velocity (mm/s): 105000


In [25]:
theta=float(input("Impact angle (degrees):"))

Impact angle (degrees): 90


In [26]:
particlemass_kg=(length*width/partarea)*peeningtime*massflowrate_kg
particlemass_mt=particlemass_kg/1000
particlemass_kg

0.0009623217922606923

In [27]:
jobname=input("Job Name: (No Spaces, Cannot begin with number)")
jobname

Job Name: (No Spaces, Cannot begin with number) CW32_0945_20_49100_105000_90


'CW32_0945_20_49100_105000_90'

In [28]:
striptype=input("Strip Type: (N, A, or C)")
striptype

Strip Type: (N, A, or C) A


'A'

In [29]:
'''
dimplesize=0.0439

n_impacts= int(length*width*coverage/dimplesize)
n_impacts
'''

'\ndimplesize=0.0439\n\nn_impacts= int(length*width*coverage/dimplesize)\nn_impacts\n'

In [30]:
'''
impacts=random.sample(range(0, len(particledata)-1), n_impacts)
#impacts
'''

'\nimpacts=random.sample(range(0, len(particledata)-1), n_impacts)\n#impacts\n'

In [31]:
density=7.98*10**-9
density

7.980000000000001e-09

In [32]:
rollingmass=0
m=0
impacts=np.empty((0,1))
while rollingmass < particlemass_mt:
    impacts=np.append(impacts,random.sample(range(0, len(particledata)-1), 1))
    radius=areaeqdiameter[int(impacts[m])]/2000
    particlemass=(4/3)*np.pi*density*radius**3
    m=m+1
    rollingmass=rollingmass+particlemass
impacts=impacts.reshape((len(impacts),1))
impacts[5]

array([1834.])

In [33]:
print("{},{},{}".format(len(impacts), rollingmass, particlemass_mt))

566,9.629842810940003e-07,9.623217922606923e-07


In [34]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [35]:
particles_call=np.empty((len(impacts),1))
for x in range (0,len(impacts),1):
    particles_call[x]=find_nearest(diametermatrix,diameters[int(impacts[x])])

In [36]:
radius=areaeqdiameter[int(impacts[5])]/2000
radius

0.32878265974350795

In [37]:
#xcoord=np.empty((len(impacts),1))
#ycoord=np.empty((len(impacts),1))
#zcoord=np.empty((len(impacts),1))
xcoord=[float(random.randint(0, 5000))/1000 for _ in range(len(impacts))]
ycoord=[float(random.randint(0, 5000))/1000 for _ in range(len(impacts))]
zcoord=[float(random.randint(1500, 3500))/1000 for _ in range(len(impacts))]
theta=[np.random.normal(loc=theta, scale=0.1) for _ in range(len(impacts))]
phi=[np.random.normal(loc=0, scale=0.1) for _ in range(len(impacts))]
#theta=[float(np.random.normal(90, sigma))/1000 for _ in range(len(impacts))]
#phi=[float(random.randint(0, 360000))/1000 for _ in range(len(impacts))]



In [38]:
header=("*Heading\n** Job name: {} Model name: {}\n** Generated by: LAF INP Generator\n*Preprint, echo=NO, model=NO, history=NO, contact=NO\n**\n** PARTS\n".format(jobname,jobname))
textheader=("x,y,AreaEqDiameter,impactdiameter\n")
f=open('{}.inp'.format(jobname),'a+')
t=open('{}.txt'.format(jobname),'a+')
f.write(header)
t.write(textheader)
for x in range (0,len(impacts),1):
    name="shot"
    radius=areaeqdiameter[int(impacts[x])]/2000
    mass=(4/3)*np.pi*density*radius**3
    #############################################################
    box=[[0,0],[0,5],[5,5],[5,0]]
    contactradius=diameters[x]/2000
    circle=Point(xcoord[x],ycoord[x]).buffer(contactradius)
    intersect=calculate_iou(box,circle)
    mass=mass*intersect
    #######################################################################################
    temp=("**\n*Part, name={}{}\n" .format(name, "{:05d}".format(x)))
    f.write(temp)
    j=int(particles_call[x])
    temp=open(r"C:\Users\lfeltner\Dropbox\CSEE\ParticleSelector\UpdatedParticles - Copy (2)\{}{}.inp".format(name, "{:02d}".format(j)),'r')
    f.write((temp.read()[22:]))
    temp="*ELSET, ELSET=E{}, GENERATE\n 1,     {} \n*Node\n      1,           0.,           0.,           0.\n*Nset, nset=shot{}-RefPt_, internal\n1, \n*Nset, nset=Set-1\n 1,\n*Rigid Body, ref node=shot{}-RefPt_, ELSET=E{}\n*Element, type=MASS, elset=Set-1_Inertia-1_\n{}, 1\n*Mass, elset=Set-1_Inertia-1_\n{}, \n".format("{:05d}".format(x),"{:04d}".format(int(totalelements[int(particles_call[x])-1])), "{:05d}".format(x),"{:05d}".format(x),"{:05d}".format(x),"{:04d}".format(int(totalelements[int(particles_call[x])-1])),'{:.20f}'.format(mass))
    f.write(temp)
    temp=("*End Part\n")
    f.write(temp)
substrate=open("{}_Subs_Part.txt".format(striptype))
f.write(substrate.read())
f.write("\n**\n**\n** ASSEMBLY\n**\n*Assembly, name={}_assembly\n**  \n*Instance, name={}_SUBS-1, part={}_SUBS\n 0,0,-1.295\n*End Instance\n".format(jobname, striptype, striptype))
for x in range (0,len(impacts),1):
    #xcoord=float(random.randint(0, 5000))/1000
    #ycoord=float(random.randint(0, 5000))/1000
    #zcoord=float(random.randint(3000, 6000))/1000
    radius=areaeqdiameter[int(impacts[x])]/2000
    #########################################################################
    adjx=xcoord[x]+((zcoord[x]-radius)*np.sin(np.radians(90-theta[x]))*np.cos(np.radians(phi[x])))
    adjy=ycoord[x]+((zcoord[x]-radius)*np.sin(np.radians(90-theta[x]))*np.sin(np.radians(phi[x])))
    adjz=(zcoord[x]-(0.65*radius))*np.cos(np.radians(90-theta[x]))
    ##adjz=(zcoord[x]+1.295-radius)*np.cos(np.radians(90-theta[x]))
    ########################################################################
    temp=("**  \n*Instance, name=shot{}, part=shot{}\n{},    {},          {}\n{},    {},          {}, {},    {},          {}, {}\n*End Instance\n" .format("{:05d}".format(x),"{:05d}".format(x), adjx, adjy, adjz, adjx, adjy, adjz, adjx, adjy+1, adjz, int(90)))
    f.write(temp)
    temp3=("{},{},{},{}\n".format(xcoord[x],ycoord[x],radius*2,float(contactradiii[int(impacts[x])]/1000)))
    t.write(temp3)
#temp=open("{}_elsets.inp".format(striptype),'r')
#f.write(temp.read())
f.write("**\n")
for x in range (0,len(impacts),1):
    temp=("*Nset, nset=RefPts, instance=shot{}\n 1,\n".format("{:05d}".format(x)))
    f.write(temp)
for x in range (0,len(impacts),1):
    temp=("*Nset, nset=SHOT{}_VeloSet, instance=shot{}\n 1,\n".format("{:05d}".format(x),"{:05d}".format(x)))
    f.write(temp)
    ##Assign velocity to each nset based on phi and theta
subssets=open("{}_Subs_Sets.txt".format(striptype))
f.write(subssets.read())
f.write("\n*End Assembly\n")
materials=open("{}_Subs_Mats.txt".format(striptype))
f.write(materials.read())
f.write("\n**\n ** PREDEFINED FIELDS \n")
for x in range (0,len(impacts),1):
    xvelo=-velo*np.sin(np.radians(90-theta[x]))*np.cos(np.radians(phi[x]))
    yvelo=-velo*np.sin(np.radians(90-theta[x]))*np.sin(np.radians(phi[x]))
    zvelo=-velo*np.cos(np.radians(90-theta[x]))
    temp=("** \n** Name: ShotVelo{}   Type: Velocity\n*Initial Conditions, type=VELOCITY\nSHOT{}_VeloSet, 1, {}\nSHOT{}_VeloSet, 2, {}\nSHOT{}_VeloSet, 3, {}\n".format("{:05d}".format(x),"{:05d}".format(x), xvelo,"{:05d}".format(x), yvelo, "{:05d}".format(x), zvelo))
    f.write(temp)
#temp=open"BCsAndSteps.inp", 'r')
#f.write(temp.read())
ints=open("{}_Subs_Ints.txt".format(striptype))
f.write(ints.read())
f.close()
t.close()
#change analytical surface to elset of all elements-rigid body definition